In [3]:
import pandas as pd
import numpy as np


План
Загрузка данных 
1. Загружаем файлы с транзакциями и клиентами
2. Пробуеем пофиксить косячные данные

Поиск транзакций связанных с кофе 
1. Транзакции где явно есть отдельностоящие слова типа кофе, кофейня или название популярного места 
2. Транзакции где в символах встречается что-то похожее на "коф" + ручной отбор таких транзакций 
3. Транзакции где в символах встречается название популярного места где продают кофе, но в названии самого места нет кофе. Например: Starbucks 
4. Транзакции в которых встречаютстя кофейны слова типа latte, cappuccino 
5. Проверка MCC кодов кофейных транзакций на адекватность. Если в каких-то транзакциях найдем MCC далекий от кофейной тематики мы отдельно ее проверим такие транзакции
6. Подготовка компактной таблицы с результатами

Выделение клиентов
1. Считаем сколько каждый клиент в последние месяцы делал кофейных транзакций 
2. Выбираем из таблицы клиентов, у кого в любых двух месяцах было 2 или больше кофейных покупок 
---------------------------------------------------------


1. Загружаем файлы с транзакциями и клиентами

In [4]:
df_clients = pd.read_csv("./clients_small.csv", sep=';')
df_transactions = pd.read_csv("./transactions_small.csv", sep=';')


In [5]:
df_clients.head(3)

,customer_id,card_id,region_name,sex,birth_date
0,02F4DB34C5696A668068CA3C1DE7C2D5,6600B4FDA036A6F00D464541EAD3B21F,Москва,F,1938-08-18
1,0308713D02D676920F2961E2A84B4516,1FB9344B8661069910D184EF19C448C7,Сахалинская область,F,1990-03-26
2,032672FDC2CF85213079F83E447AF75B,4C8AD60064AF412753EB54EAFCD8C59D,Самарская область,F,1989-01-13


In [6]:
df_transactions.head(3)

,customer_id,card_id,tr_datetime,tr_normalized_amount,currency,mcc,merchant_name,merchant_city
0,634E7564849D27B3C452229E405629A6,52CD1B87531555F5611C663562FC686D,2019-09-07,10.00,810,5999,i-bank ┴шыaщэ >moscow ru,moscow
1,B20A638E7EB926C63102144596F38C71,9D4F087597B65028953FBE8A8FBF48DE,2019-09-20,313.04,810,5411,dixy moskva ru,moscow
2,125EBE0A38873D4888408F85877282EF,B8859E8DB421C73B785521A6252A36CB,2019-09-23,250.00,810,5999,i-bank teыe 2 >moscow ru,moscow


2. Пробуеем пофиксить косячные данные

Пробуем найти косячные данные типа "i-bank ┴шыaщэ" в столбце merchant_name \
Для этого создаем функцию, которая находит не Unicode символы 

In [7]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [8]:
df_transactions['merchant_name_eng_name'] = df_transactions.apply(lambda x: isEnglish(x['merchant_name']), axis=1)

Исследуем merchant_name которые на распознались и оцениваем их количество в общей выборке

In [9]:
df_transactions[df_transactions['merchant_name_eng_name'] == False ]['merchant_name'].unique()

array(['i-bank ┴шыaщэ         >moscow         ru',
       'i-bank teыe 2         >moscow         ru',
       'i-bank meуa╘oэ        >moscow         ru',
       'i-bank meгaфoн        >moscow         ru',
       'i-bank билaйн         >moscow         ru',
       'i-bank ╪ъoы№эoe яшєaэш>moscow         ru',
       'i-bank teлe 2         >moscow         ru',
       'i-bank m├tc           >moscow         ru',
       'i-bank ─om.ru         >moscow         ru',
       'i-bank teыe 2 (aтєoяыa>moscow         ru',
       'i-bank yota. coєoтa\xa0 c>moscow         ru',
       'i-bank mгtc           >moscow         ru',
       'i-bank mtc (aтєoяыaєeц>moscow         ru',
       'i-bank moєшт          >moscow         ru',
       'i-bank pocєeыeъoь ╧ыaє>moscow         ru',
       'i-bank tpшъoыop tb    >moscow         ru',
       'i-bank ┴шыaщэ (aтєoяыa>moscow         ru',
       'i-bank aшcє           >moscow         ru',
       'i-bank дom.ru         >moscow         ru',
       'i-bank mtc фикcиpoвa

In [10]:
bad_merchants =  df_transactions[df_transactions['merchant_name_eng_name'] == False ]['merchant_name'].shape[0]
good_merchants =  df_transactions['merchant_name'].shape[0]
print('good_merchants : ',good_merchants,'| bad_merchants : ',bad_merchants,'| bad_merchants_percent : ', 100*bad_merchants/good_merchants)

good_merchants :  623956 | bad_merchants :  15670 | bad_merchants_percent :  2.5113950342652367


Процент таких мерчантов не такой большой \
По большей части из них можно понять что за платеж 'i-bank taкcи maкcим'\
По меньшей части не совсем понятно 'i-bank ▀эфeъc.─eэ№уш'\
Пробовал меньшую часть расшифровывать через сервисы подбора кодировки, но ничего не вышло. \
Пробовал отдельно расшифровать через разные кодировки но ничего не вышло \
По большей части глазами пробовал найти кофейную тематику, но ничего не нашел \
По меньшей части просто забил.  Даже если в этой части мерчантов были операции с кофе в аналитике это не будет отражено.\
Но судя по тому что это операции в интернет банке, вряд ли там есть покупки кофе. 


In [11]:
df_transactions[df_transactions['merchant_name_eng_name'] == False ]['merchant_name'].unique()

array(['i-bank ┴шыaщэ         >moscow         ru',
       'i-bank teыe 2         >moscow         ru',
       'i-bank meуa╘oэ        >moscow         ru',
       'i-bank meгaфoн        >moscow         ru',
       'i-bank билaйн         >moscow         ru',
       'i-bank ╪ъoы№эoe яшєaэш>moscow         ru',
       'i-bank teлe 2         >moscow         ru',
       'i-bank m├tc           >moscow         ru',
       'i-bank ─om.ru         >moscow         ru',
       'i-bank teыe 2 (aтєoяыa>moscow         ru',
       'i-bank yota. coєoтa\xa0 c>moscow         ru',
       'i-bank mгtc           >moscow         ru',
       'i-bank mtc (aтєoяыaєeц>moscow         ru',
       'i-bank moєшт          >moscow         ru',
       'i-bank pocєeыeъoь ╧ыaє>moscow         ru',
       'i-bank tpшъoыop tb    >moscow         ru',
       'i-bank ┴шыaщэ (aтєoяыa>moscow         ru',
       'i-bank aшcє           >moscow         ru',
       'i-bank дom.ru         >moscow         ru',
       'i-bank mtc фикcиpoвa

Будем искать транзакции, где клиенты покупали кофе/посещали кофейни.\
Так как мы не знаем, что конкретно покупал клиент в точке будет подбирать те точки, где больше половины ассортимента это кофе.\ 
При этом будет понимать, что в нашу выборку могут попасть и те клиенты, кто "зашел в Старбакс и купил бутылку воды".\
Будем отбирать в 6 стадий:\
1. Транзакции где явно есть отдельностоящие слова типа кофе, кофейня или название популярного места 
2. Транзакции где в символах встречается что-то похожее на "коф" + ручной отбор таких транзакций 
3. Транзакции где в символах встречается название популярного места где продают кофе, но в названии самого места нет кофе. Например: Starbucks 
4. Транзакции в которых встречаютстя кофейны слова типа latte, cappuccino 
5. Проверка MCC кодов кофейных транзакций на адекватность. Если в каких-то транзакциях найдем MCC далекий от кофейной тематики мы отдельно ее проверим такие транзакции
6. Подготовка компактной таблицы с результатами
----------------------------------


1. Транзакции где явно есть отдельностоящие слова типа кофе, кофейня или название популярного места 

In [12]:
list_exact_coffee = df_transactions[
    (df_transactions['merchant_name'].str.contains('coffee', case=False,regex=False)) | 
    (df_transactions['merchant_name'].str.contains(r'\b(?:coffe|cofe|kofe|koffe|koffee|coffe|cofee|kofejnya|kofeiynya|kofeynya|kofeinya|kofeinia|kofeshop|kofeinaya|kofejnaya|cofix|kofein|onepricecoffe|koffein|kofeynyy)\b', case=False,regex=True)) 
]['merchant_name'].unique().tolist()

In [13]:
len(list_exact_coffee)

961

2. Транзакции где в символах встречается что-то похожее на "коф" + ручной отбор таких транзакций 

In [14]:
df_transactions_prob_coffee = df_transactions[
    (df_transactions['merchant_name'].str.contains('cof|kof', case=False,regex=True)) &
    (~df_transactions['merchant_name'].isin(list_exact_coffee)) 
    ]['merchant_name'].unique()

Выгрузим файл мерчанты которые похожи на кофе, чтобы разметить их вручную. Их чуть больше 200, что не так много. 

In [15]:
pd.DataFrame(df_transactions_prob_coffee).to_excel('df_transactions_prob_coffee.xlsx')

In [16]:
# тут я поработал с Excel файлом
# руками разметил мерченты которые явно относятся к кофе 

In [17]:
df_transactions_prob_coffee = pd.read_excel('df_transactions_prob_coffee_fixed.xlsx',index_col = 0)

In [18]:
df_transactions_prob_coffee.columns = ['merchant_name', 'coffee']


In [19]:
#добавляем к списку наших кофейных мерчантов тех, которых мы отобрали вручную 

list_exact_coffee = list_exact_coffee + df_transactions_prob_coffee[df_transactions_prob_coffee['coffee'] == 1]['merchant_name'].to_list()

In [20]:
len(list_exact_coffee)

1113

3. Транзакции где в символах встречается название популярного места где продают кофе, но в названии самого места нет кофе. Например: Starbucks \
Погуглил в интернете кофейни и магазины кофе и искал то что без "Коф" в названии
Также пользовался вот этим списком

In [21]:
list_exact_coffee = list_exact_coffee + df_transactions[df_transactions['merchant_name'].str.contains('starbucks|bodryj den|shokoladnitsa|doubleb|bloom n brew|cooperative chernyi', case=False,regex=True)]['merchant_name'].to_list()

4. Транзакции в которых встречаютстя кофейны слова типа latte, cappuccino 

In [22]:
list_exact_coffee = list_exact_coffee + df_transactions[df_transactions['merchant_name'].str.contains('capuchino|cappuccino|latte|robusta|espresso', case=False,regex=True)]['merchant_name'].to_list()

In [23]:
len(list_exact_coffee)

1881

In [24]:
#удаляем дубликаты из листа мерчантов
list_exact_coffee = list(dict.fromkeys(list_exact_coffee))

In [25]:
len(list_exact_coffee)

1314

Размечаем транзакции в которых есть покупки кофе 

In [26]:
df_transactions['coffee_transaction'] = np.where(df_transactions['merchant_name'].isin(list_exact_coffee), True, False)

5. Проверка MCC кодов кофейных транзакций на адекватность. Если в каких-то транзакциях найдем MCC далекий от кофейной тематики мы отдельно ее проверим такие транзакции\
Пользовался вот этим справочником https://merchantpoint.ru/mcc

In [27]:
df_transactions[df_transactions['coffee_transaction'] == True]['mcc'].unique()

array([5814, 5812, 5411, 5813, 5499, 5999, 5441, 7230, 5399, 8999, 5993,
       5331, 5969, 5462, 5811, 7832, 5661, 5992, 7999, 5944, 4812, 5533,
       5311, 5651], dtype=int64)

Нашли подозрительные категории: \
7230 - Парикмахерские и салоны красоты \
5969 - Прямой маркетинг – другие торговые точки прямого маркетинга (нигде более не классифицированные) \
7832 - Кинотеатры \
5661 - Обувные магазины	\
5922 - Флористика \
5944 - Часы, ювелирные и серебряные изделия \
4812 - Телекоммуникационное оборудование, включая продажу телефонов \
5533 - Автозапчасти и аксессуары \
5651	Одежда для всей семьи \
\
Изучим транзакции в этих категориях, если транзакция не похожа нк кофе, то выкинем.  

In [28]:
df_transactions[
    (df_transactions['coffee_transaction']) &
    (df_transactions['mcc'].isin([7230,5969,7832,5661,5922,5944,4812,5533,5651]))]

,customer_id,card_id,tr_datetime,tr_normalized_amount,currency,mcc,merchant_name,merchant_city,merchant_name_eng_name,coffee_transaction
14377,7C1C451DB2510B295F77E2CC2AC5F8DD,AA00F803B05F654B720682565CB91B90,2019-10-13,600.0,810,7230,"""""""coffee & moloko"""" salo sankt-peterbru""",saint petersburg,True,True
91517,C841189CE436A41D06676E25A46B7BB2,A4596B479BC539425A7E34D46504ECDC,2019-10-10,3030.0,810,5969,nespresso moskva ru,moscow,True,True
166668,6DA2F441CDE44CF83C6EA70D15F39E1F,24DE1B66B0DF8594C586D012E51102A4,2019-08-25,1000.0,810,5969,nespresso kotelniki ru,kotelniki,True,True
246291,8C0BCAE7DB8926ACC47F915539C51142,8F66075E4A8705DB7F3AB72E2F7085DC,2019-10-31,160.0,810,7832,kofe ulan-ude ru,ulan-ude,True,True
258224,C23E7EBA5725AF6993AB1A4CB948F7B4,9F8A3653462E676F24BA1DBCBDB98121,2019-10-15,438.0,810,5661,surf coffee petropavlovsru,petropavlovs,True,True
361741,8F5084DBEFA1F902BEA27C68B45FB447,B5437DEDF66B3F39D0D1E7E0FB88476B,2019-08-23,960.0,810,5969,nespresso moskva ru,moscow,True,True
368127,7E3D601079B3A15574DF1BCDBE7D4DFA,36D354167A5F241C532C08EB2E7AB68C,2019-08-01,147.0,810,5944,chaykoffskiy severodvinskru,severodvinsk,True,True
403512,592D9AFAB685D89E502C012001873ED1,408BD48A186180DA73224CC223E77556,2019-09-25,2201.0,810,4812,travelers coffee ulan-ude ru,ulan-ude,True,True
415024,8C0BCAE7DB8926ACC47F915539C51142,8F66075E4A8705DB7F3AB72E2F7085DC,2019-10-31,290.0,810,7832,kofe ulan-ude ru,ulan-ude,True,True
459339,592D9AFAB685D89E502C012001873ED1,408BD48A186180DA73224CC223E77556,2019-09-30,1397.0,810,4812,travelers coffee ulan-ude ru,ulan-ude,True,True


Судя по транзакциям и гуглению в интеренете все эти точки действительно продают кофе. \
Видимо просто не совсем верно указали MCC коды

In [29]:
df_transactions.head(3)

,customer_id,card_id,tr_datetime,tr_normalized_amount,currency,mcc,merchant_name,merchant_city,merchant_name_eng_name,coffee_transaction
0,634E7564849D27B3C452229E405629A6,52CD1B87531555F5611C663562FC686D,2019-09-07,10.00,810,5999,i-bank ┴шыaщэ >moscow ru,moscow,False,False
1,B20A638E7EB926C63102144596F38C71,9D4F087597B65028953FBE8A8FBF48DE,2019-09-20,313.04,810,5411,dixy moskva ru,moscow,True,False
2,125EBE0A38873D4888408F85877282EF,B8859E8DB421C73B785521A6252A36CB,2019-09-23,250.00,810,5999,i-bank teыe 2 >moscow ru,moscow,False,False


6. Подготовка компактной таблицы с результатами

In [30]:
date = pd.to_datetime(df_transactions['tr_datetime'])
df_transactions['month'] = date.dt.month

In [31]:
df_transactions_short = df_transactions[df_transactions['coffee_transaction'] == True][['customer_id','month','card_id']] 

In [32]:

df_transactions['customer_id'].unique().shape[0]

5370

Выделение клиентов
1. Считаем сколько каждый клиент в последние месяцы делал кофейных транзакций 
2. Выбираем из таблицы клиентов, у кого в любых двух месяцах было 2 или больше кофейных покупок
-----------------------------

1. Считаем сколько каждый клиент в последние месяцы делал кофейных транзакций 

In [33]:
df_coffee_pivot =  df_transactions_short.pivot_table(columns=['month'],index=['customer_id'],aggfunc= 'count',fill_value=0).reset_index()

In [34]:
df_coffee_pivot.shape[0]

1151

In [35]:
#Фиксим колонки в табличке от мульти-индекса
df_coffee_pivot.columns = [' '.join(map(str, col)).strip() for col in df_coffee_pivot.columns.values]

In [36]:
df_coffee_pivot.columns = ['customer_id', '8', '9', '10']

In [37]:
df_coffee_pivot.head(3)

,customer_id,8,9,10
0,003E71F5DF267A7F73CFD9B1E706D554,0,1,0
1,0060525A49B72C438F891AFF736713E5,1,0,0
2,01101BD054492A27D0312CA2F1DBA42F,0,1,0


2. Выбираем из таблицы клиентов, у кого в любых двух месяцах было 2 или больше кофейных покупок

In [38]:
df_coffee_pivot[
    ((df_coffee_pivot['8']>1) & (df_coffee_pivot['9']>1)) |
    ((df_coffee_pivot['8']>1) & (df_coffee_pivot['10']>1)) |
    ((df_coffee_pivot['9']>1) & (df_coffee_pivot['10']>1))]

,customer_id,8,9,10
5,01F4759D39152AD450EB146328553FF4,4,2,0
9,02742357300A71889E018AD35941DA09,2,2,0
12,03FF8D5865069ED9A1810F0464AD0FBA,11,5,5
13,0405815580CA4D1A190D4E04DA422485,10,3,7
14,040CA8756FC68D5D173495B66F8B0C28,3,3,2
...,...,...,...,...
1137,FDF18BD8668193BB7EA965A1353E1FD0,1,3,2
1138,FE257FE01D64EDA62D336934C01756B7,6,0,2
1142,FE9698DE39B77B5DB9EAFE1D9B13CE67,2,4,0
1143,FEB266B829BAB63CA5CD60CB73B74877,5,3,3


Откровенно говоря мы нашли не совсем то что требовалось в задании. \
\
Требовалось "Выделить клиентов, которые покупали кофе/посещали кофейни от 2 раз в месяц хотя бы 2 из 3 месяцев." \
Мы тут сделали "Выделить клиентов, которые посещали точки продажи кофе /посещали кофейни от 2 раз в месяц хотя бы 2 из 3 месяцев." \
Например, есть магазин 'Кофейная кантата' и если у клиента есть покупка там, то мы засчитываем его за человека который покупал кофе/посещал кофейню. \
Хотя на самом деле он мог купить там чай. \
\
Для того чтобы выделить людей, кто покупал кофе нужны данные по купленным товарам в разрезе транзакций. \
Если они будут, то смогу взять тех кто покупал именно кофе, а не посещал место где активно продается кофе :)\    